<a href="https://colab.research.google.com/github/Bkankim/Competition/blob/main/Byeonghyeon_Competition_ML_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 준비 단계

In [ ]:
# 시각화 및 폰트 설정
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib.font_manager as fm
from tqdm import tqdm
from scipy.spatial import cKDTree
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from lightgbm import LGBMRegressor
from lightgbm import early_stopping, log_evaluation
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


import pickle
import warnings;warnings.filterwarnings('ignore')
import eli5
from eli5.sklearn import PermutationImportance

fe = fm.FontEntry(fname='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', name='NanumBarunGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'})
plt.rc('font', family='NanumBarunGothic')


## 1. 데이터 로드 및 결측치 처리

In [ ]:
# 경로 설정
train_path = '/data/ephemeral/home/Bkan/Competition_dataset/train.csv'
test_path = '/data/ephemeral/home/Bkan/Competition_dataset/test.csv'

# 데이터 로딩
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# 테스트셋 구분 컬럼 추가
train['is_test'] = 0
test['is_test'] = 1

# 전처리를 위한 통합 데이터 생성
concat = pd.concat([train, test], axis=0)

In [ ]:
# 100만개 이상 결측된 컬럼 목록
threshold = 1_000_000
cols_to_drop = missing[missing >= threshold].index.tolist()
# 삭제
cols_to_drop

In [ ]:
# 남은 변수 분리
continuous_cols = []
categorical_cols = []

for col in concat.columns:
    if pd.api.types.is_numeric_dtype(concat[col]):
        continuous_cols.append(col)
    else:
        categorical_cols.append(col)

# 특수처리: 숫자지만 범주형인 '본번', '부번'
for col in ['본번', '부번']:
    if col in continuous_cols:
        concat[col] = concat[col].astype(str)
        continuous_cols.remove(col)
        categorical_cols.append(col)

# 결측치 채움
concat[categorical_cols] = concat[categorical_cols].fillna('NULL')
concat[continuous_cols] = concat[continuous_cols].interpolate(method='linear', axis=0)

In [ ]:
# 결측치는 아닌데 의미 없는 형식적 값 찾기
def detect_fake_nulls(df, suspect_values=['-', ' ', '', '.', '없음', 'nan']):
    result = {}
    for col in df.columns:
        if concat[col].dtype == 'object':
            val_counts = concat[col].value_counts(dropna=False)
            found = val_counts[val_counts.index.isin(suspect_values)]
            if not found.empty:
                result[col] = found
    return result

fake_nulls = detect_fake_nulls(concat)

In [ ]:
# 의미 없는 값 정의
fake_null_values = ['-', ' ', '', '.', '없음', 'nan']

# 문자형 컬럼 기준 일괄 변환
object_cols = concat.select_dtypes(include='object').columns.tolist()
for col in object_cols:
    concat[col] = concat[col].replace(fake_null_values, np.nan)

In [ ]:
# 복사본 생성
concat_select = concat.copy()

# 본번, 부번 → 문자형 변환
concat_select['본번'] = concat_select['본번'].astype(str)
concat_select['부번'] = concat_select['부번'].astype(str)

# 연속형 변수 추출
continuous_cols = concat_select.select_dtypes(include=['int64', 'float64']).columns.tolist()
continuous_cols = [col for col in continuous_cols if col != 'is_test']  # is_test 제외

# 범주형 변수 추출
categorical_cols = concat_select.select_dtypes(include=['object']).columns.tolist()

In [ ]:
# 현재 수치형 변수 목록
train_only = concat[concat['is_test'] == 0].copy()
numeric_cols = train_only.select_dtypes(include=np.number).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['is_test']]

# 결측치 비율 계산
missing_ratio = train_only[numeric_cols].isnull().mean()

# 상관계수 계산
corr_matrix = train_only[numeric_cols].corr()
target_corr = corr_matrix['target'].drop('target')

# 결측비율 + 상관계수 조합 테이블
value_check = pd.DataFrame({
    '결측비율': missing_ratio,
    'target_상관계수': target_corr,
    '절대_상관': target_corr.abs()
}).sort_values(by='절대_상관', ascending=False)

In [ ]:
# 수치형 변수 필터링
train_only = concat[concat['is_test'] == 0].copy()
numeric_cols = train_only.select_dtypes(include=np.number).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['is_test']]

# 상관계수 계산
corr = train_only[numeric_cols].corr()

## 2. 이상치 처리
> 클리핑, 로그변환

In [ ]:
## 전용면적 이상치 클리핑 셀
# 복사본 생성
concat['전용면적_clip'] = concat['전용면적(㎡)'].copy()

# 상하위 0.5% 경계 계산
q_low = concat['전용면적_clip'].quantile(0.005)
q_high = concat['전용면적_clip'].quantile(0.995)

# 클리핑 적용
concat['전용면적_clip'] = concat['전용면적_clip'].clip(lower=q_low, upper=q_high)

In [ ]:
## k-주거전용면적과 k-전체세대수 사이에 상관관계가 있는지 확인
# # 세대당 전용면적 파생 변수 생성
concat['세대당_전용면적'] = concat['k-주거전용면적'] / concat['k-전체세대수']

## lgbm을 사용할 거기 때문에 로그스케일대신 클리핑을 채택
# 하위/상위 0.5% 경계 계산
low = concat['세대당_전용면적'].quantile(0.005)
high = concat['세대당_전용면적'].quantile(0.995)

# 클리핑 적용
concat['세대당_전용면적_clip'] = concat['세대당_전용면적'].clip(lower=low, upper=high)

In [ ]:
## 각별한 이상치가 없으므로 로그 변환 채택
# 로그 변환
concat['주차대수_log'] = np.log1p(concat['주차대수'])

In [ ]:
## 클리핑 분포도 이쁜편이고, 상관계수도 훨씬 높기때문에, 클리핑으로 채택
# 클리핑 경계 설정
lower = concat['k-연면적'].quantile(0.005)
upper = concat['k-연면적'].quantile(0.995)

# 클리핑 적용
concat['k-연면적_clipped'] = concat['k-연면적'].clip(lower=lower, upper=upper)

In [ ]:
## 분포가 이쁘진 않지만 로그 변환으로 해석을 어렵게 할 필요는 없어보이므로 이것도 클리핑
# 기본 통계 및 결측/상관 정보
col = 'k-주거전용면적'

# 클리핑 경계 계산 (0.5% ~ 99.5%)
low, high = np.percentile(concat['k-주거전용면적'], [0.5, 99.5])

# 클리핑 적용
concat['k-주거전용면적_clipped'] = concat['k-주거전용면적'].clip(lower=low, upper=high)

In [ ]:
# concat에서 만든 파생변수들을 concat_select에 모두 복사
concat_select['전용면적_clip'] = concat['전용면적_clip']
concat_select['세대당_전용면적'] = concat['세대당_전용면적']
concat_select['세대당_전용면적_clip'] = concat['세대당_전용면적_clip']
concat_select['주차대수_log'] = concat['주차대수_log']
concat_select['k-연면적_log'] = concat['k-연면적_log']
concat_select['k-연면적_clipped'] = concat['k-연면적_clipped']
concat_select['k-주거전용면적_clipped'] = concat['k-주거전용면적_clipped']

In [ ]:
# 도메인 기반 사고 & 결측치도 동반하기 때문에 필요없다 판단하여 삭제
rop_cols = ['등기신청일자', '중개사소재지', '거래유형', '도로명', 'k-시행사']
concat_select = concat_select.drop(columns=drop_cols)

In [ ]:
# 컬럼 동기화 (드롭된 컬럼 제거)
categorical_cols = [col for col in categorical_cols if col in concat_select.columns]
continuous_cols = [col for col in continuous_cols if col in concat_select.columns]

## 3. 파생 변수 생성
> 따릉이, 지하철역 거리
>
> KDTree : 브루트포스보다 훨씬 빠른 처리 및 아파트와의 직선거리 계산

In [ ]:
# 시군구, 년월 등 분할할 수 있는 변수들은 세부사항 고려를 용이하게 하기 위해 모두 분할해 주겠습니다.
concat_select['구'] = concat_select['시군구'].map(lambda x : x.split()[1])
concat_select['동'] = concat_select['시군구'].map(lambda x : x.split()[2])
del concat_select['시군구']

concat_select['계약년'] = concat_select['계약년월'].astype('str').map(lambda x : x[:4])
concat_select['계약월'] = concat_select['계약년월'].astype('str').map(lambda x : x[4:])
del concat_select['계약년월']

In [ ]:
# 강남의 여부를 체크합니다.
is_gangnam = []
for x in concat_select['구'].tolist() :
  if x in gangnam :
    is_gangnam.append(1)
  else :
    is_gangnam.append(0)

# 파생변수를 하나 만릅니다.
concat_select['강남여부'] = is_gangnam

In [ ]:
## 베이스코드 feature importance와 내 도메인 지식을 기반으로 제거할 컬럼을 정해 제거함.
drop_indices = [8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 21, 26, 27, 28, 29, 32, 33, 34, 35, 38]
drop_cols = [concat_select.columns[i] for i in drop_indices]
concat_select.drop(columns=drop_cols, inplace=True)

In [ ]:
# 외부 파일 불러오기
subway = pd.read_csv('/data/ephemeral/home/Bkan/Competition_dataset/subway_feature.csv')  # 지하철역 위경도 데이터
bike = pd.read_csv('/data/ephemeral/home/Bkan/Competition_dataset/bike_station.csv', encoding='cp949')      # 따릉이 대여소 위경도 데이터

In [ ]:
# 아파트 기준 좌표 (위도, 경도)
apt_coords = np.vstack([concat_select['좌표Y'], concat_select['좌표X']]).T

# 지하철 좌표
subway_coords = np.vstack([subway['위도'], subway['경도']]).T
subway_tree = cKDTree(subway_coords)

# 따릉이 좌표 (결측/0.0 제거)
bike_clean = bike[(bike['위도'] > 0) & (bike['경도'] > 0)].copy()
bike_coords = np.vstack([bike_clean['위도'], bike_clean['경도']]).T
bike_tree = cKDTree(bike_coords)

In [ ]:
# KDTree를 위한 기준 좌표 준비
apt_coords = np.vstack([concat_select['좌표Y'], concat_select['좌표X']]).T

# 지하철 KDTree
subway_coords = np.vstack([subway['위도'], subway['경도']]).T
subway_tree = cKDTree(subway_coords)

# 따릉이 KDTree (0.0 또는 NaN 제거)
bike_clean = bike[(bike['위도'] > 0) & (bike['경도'] > 0)]
bike_coords = np.vstack([bike_clean['위도'], bike_clean['경도']]).T
bike_tree = cKDTree(bike_coords)

# 최단거리
concat_select['지하철_최단거리'] = subway_tree.query(apt_coords)[0]
concat_select['따릉이_최단거리'] = bike_tree.query(apt_coords)[0]

# 반경 내 지하철역 개수 (500m ≈ 0.005도)
subway_counts = subway_tree.query_ball_point(apt_coords, r=0.005)
concat_select['지하철_500m내_개수'] = [len(x) for x in subway_counts]

# 반경 내 따릉이 대여소 개수 (300m ≈ 0.003도)
bike_counts = bike_tree.query_ball_point(apt_coords, r=0.003)
concat_select['따릉이_300m내_개수'] = [len(x) for x in bike_counts]

In [ ]:
## 분포가 오른쪽 긴꼬리 양상을 보이기 때문에 로그 변환 실행
# 로그 변환 적용
concat_select['지하철_최단거리_log'] = np.log1p(concat_select['지하철_최단거리'])
concat_select['따릉이_최단거리_log'] = np.log1p(concat_select['따릉이_최단거리'])

## 4. 모델 정의 및 학습 실행
> LightGBM 채택
>
> 1. 속도 : RF 대비 10배 빠른 처리 속도
>
> 2. 도메인 적합성 : 부동산의 복잡한 비선형성에 최적
>
> 3. 성능 : 95% 이상 향상 (실제 대회 결과)

In [ ]:
# 학습용과 테스트용 분리
df = concat_select.copy()
train_df = df[df['is_test'] == 0]
X = train_df.drop(columns=['target', 'is_test'])
y = train_df['target']

# 학습 / 검증셋 분할
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# lgbm은 한글 변수를 학습하지 못하기 때문에 인코딩 작업
cat_cols_to_encode = [
    '번지', '본번', '부번', '아파트명',
    'k-난방방식', 'k-수정일자',
    '구', '동', '계약년', '계약월'
]

# 전체 데이터(train + test)를 사용하여 LabelEncoder 학습
label_encoders = {}
for col in cat_cols_to_encode:
    le = LabelEncoder()
    # 전체 데이터를 사용하여 fit (train + test 모두 포함)
    le.fit(df[col].astype(str))
    label_encoders[col] = le

    # train과 valid에 transform 적용
    X_train[col] = le.transform(X_train[col].astype(str))
    X_valid[col] = le.transform(X_valid[col].astype(str))

In [ ]:
# 데이터셋 변환
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train, free_raw_data=False)

# 하이퍼파라미터 기본값
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,  # 기본값
    'learning_rate': 0.05,  # 기본값
    'feature_fraction': 0.9,  # 약간만 조정
    'verbosity': -1,
    'seed': 42,
}

# 학습 실행
model = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_valid],
    num_boost_round=4000,
    callbacks=[
        early_stopping(100),
        log_evaluation(100)  # 100번째마다 로그 출력
    ]
)